<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/SQLAlchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [223]:
import sqlite3
from tabulate import tabulate

db = 'chat.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        # print(results)
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+----------+
|   id |   name   |
+======+==========+
|    1 | John Doe |
+------+----------+


chats
+------+-----------+--------+--------------+
|  id  |  user_id  |  name  |  created_at  |
+======+===========+========+==============+
+------+-----------+--------+--------------+


messages
+------+-----------+-----------+-----------+--------------+
|  id  |  user_id  |  chat_id  |  message  |  created_at  |
+======+===========+===========+===========+==============+
+------+-----------+-----------+-----------+--------------+




In [224]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, Text, ForeignKey, delete, update, and_
import datetime

engine = create_engine("sqlite:///chat.db")


metadata = MetaData()


users_table = Table(
    "users",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(255)),
)

chats_table = Table(
    "chats",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("users.id", ondelete="cascade")),
    Column("name", String(255)),
    Column("created_at", DateTime, default=datetime.datetime.utcnow()),
)

messages_table = Table(
    "messages",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("users.id", ondelete="cascade")),
    Column("chat_id", Integer, ForeignKey("chats.id", ondelete="cascade")),
    Column("message", Text),
    Column("created_at", DateTime, default=datetime.datetime.utcnow()),
)


metadata.create_all(engine)


def add_user(name):
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == name)
        ).fetchone()
        if user is not None:
            return
        else:
            user = {"name": name}
            connection.execute(users_table.insert(), [user])
            connection.commit()


def add_chat(user_name, chat_name):
    with engine.connect() as connection:
        chat = connection.execute(
            chats_table.select().where(
                and_(
                    chats_table.c.name == chat_name,
                    chats_table.c.user_id == connection.execute(
                        users_table.select().where(users_table.c.name == user_name)
                    ).fetchone().id,
                )
            )
        ).fetchone()
        if chat is not None:
            return
        else:
            user = connection.execute(
                users_table.select().where(users_table.c.name == user_name)
            ).fetchone()
            chat = {"name": chat_name, "user_id": user.id}
            connection.execute(chats_table.insert(), [chat])
            connection.commit()


def add_message(user_name, chat_name, message):
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == user_name)
        ).fetchone()
        chat = connection.execute(
            chats_table.select().where(chats_table.c.name == chat_name)
        ).fetchone()
        message = {"chat_id": chat.id, "user_id": user.id, "message": message}
        connection.execute(messages_table.insert(), [message])
        connection.commit()


def delete_user(name):
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == name)
        ).fetchone()
        if user is None:
            return
        else:
            connection.execute(delete(users_table).where(users_table.c.name == name))
            connection.execute(delete(chats_table).where(chats_table.c.user_id == user.id))
            connection.execute(delete(messages_table).where(messages_table.c.user_id == user.id))
            connection.commit()


def delete_chat(user_name, chat_name):
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == user_name)
        ).fetchone()
        if user is None:
            return
        else:
            chat = connection.execute(
                chats_table.select().where(chats_table.c.name == chat_name)
            ).fetchone()
            if chat is None:
                return
            else:
                connection.execute(
                    delete(chats_table).where(chats_table.c.id == chat.id)
                )
                connection.execute(
                    delete(messages_table).where(messages_table.c.chat_id == chat.id)
                )
                connection.commit()


def delete_message(user_name, chat_name, message):
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == user_name)
        ).fetchone()
        if user is None:
            return
        else:
            chat = connection.execute(
                chats_table.select().where(chats_table.c.name == chat_name)
            ).fetchone()
            if chat is None:
                return
            else:
                message = connection.execute(
                    messages_table.select().where(
                        and_(
                            messages_table.c.user_id == user.id,
                            messages_table.c.chat_id == chat.id,
                            messages_table.c.message == message,
                        )
                    )
                ).fetchone()
                if message is not None:
                    connection.execute(
                        delete(messages_table).where(messages_table.c.id == message.id)
                    )
                    connection.commit()



# Пример использования методов
# add_user("John Doe")
# add_chat("John Doe", "My Chat")
# add_message("John Doe", "My Chat", "Hello, world!")
# delete_user("John Doe")
# delete_chat("John Doe", "My Chat")
# delete_message("John Doe", "My Chat", "Hello, world!")